If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [1]:
! pip install datasets transformers seqeval accelerate==0.29.3

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import transformers

print(transformers.__version__)

4.38.2


# Fine-tuning a model on a token classification task

In [14]:
task = "ner"
dataset_path = 'custom_dataset_track1'
base_model = 'PlanTL-GOB-ES/roberta-base-biomedical-clinical-es'
model_checkpoint = "PlanTL-GOB-ES/roberta-base-biomedical-clinical-es"
output_path = 'output/models/'

In [4]:
batch_size = 32

## Loading the dataset

In [5]:
from datasets import load_dataset, load_metric

In [6]:
# # If running in Colab
# from google.colab import drive
# drive.mount('/content/drive')

In [15]:
from datasets import DatasetDict

dataset_split = load_dataset(dataset_path, split=[
    'train[0%:100%]', 'validation[0%:100%]'
])

datasets = DatasetDict({'train': dataset_split[0], 'validation': dataset_split[1]})

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset custom_dataset_track1 downloaded and prepared to /home/sylvia/.cache/huggingface/datasets/custom_dataset_track1/customconll2003/1.0.0/60dc2199add7f00a97849f7717af103a73c99e29a5e0e865ff1463ef003a2561. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'labels'],
        num_rows: 5182
    })
    validation: Dataset({
        features: ['id', 'tokens', 'labels'],
        num_rows: 7208
    })
})

In [17]:
datasets["train"][0]

{'id': '0',
 'tokens': ['Paciente',
  'varón',
  'de',
  '35',
  'años',
  'con',
  'tumoración',
  'en',
  'polo',
  'superior',
  'de',
  'teste',
  'derecho',
  'hallada',
  'de',
  'manera',
  'casual',
  'durante',
  'una',
  'autoexploración',
  ',',
  'motivo',
  'por',
  'el',
  'cual',
  'acude',
  'a',
  'consulta',
  'de',
  'urología',
  'donde',
  'se',
  'realiza',
  'exploración',
  'física',
  ',',
  'apreciando',
  'masa',
  'de',
  '1cm',
  'aproximado',
  'de',
  'diámetro',
  'dependiente',
  'de',
  'epidídimo',
  ',',
  'y',
  'ecografía',
  'testicular',
  ',',
  'que',
  'se',
  'informa',
  'como',
  'lesión',
  'nodular',
  'sólida',
  'en',
  'cabeza',
  'de',
  'epidídimo',
  'derecho',
  '.',
  'Se',
  'realiza',
  'RMN',
  '.',
  'Confirmando',
  'masa',
  'nodular',
  ',',
  'siendo',
  'el',
  'tumor',
  'adenomatoide',
  'de',
  'epidídimo',
  'la',
  'primera',
  'posibilidad',
  'diagnóstica',
  '.'],
 'labels': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [10]:
label_list = datasets["train"].features[f"labels"].feature.names
label_list

['O', 'B-ENFERMEDAD', 'I-ENFERMEDAD']

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [18]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [19]:
show_random_elements(datasets["train"])

,id,tokens,labels
0,1436,"[La, evolución, semanal, de, las, Phe, a, lo, largo, de, los, primeros, 3, meses, de, seguimiento, mostraron, niveles, de, 498, ±, 15, micromoles, /, L, ., Posteriormente, la, familia, volvió, a, Argentina, donde, sigue, tratamiento, en, la, actualidad, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,340,"[Durante, su, ingreso, hospitalario, el, paciente, empieza, con, disfonía, y, disfagia, realizándose, fibroscopia, ,, observándose, lesión, polipoidea, en, seno, piriforme, ., El, TAC, cervical, normal, ., Durante, el, ingreso, al, paciente, se, le, pauta, Rituximab, y, al, día, siguiente, el, primer, ciclo, de, CHOP, (, ciclofosfamida, ,, doxorrubicina, ,, vincristina, ,, prednisona, ), ,, presentando, intensa, mejoría, de, los, síntomas, B, ,, con, desaparición, de, la, fiebre, ., Se, instaura, tratamiento, con, quimioterapia, pautándosele, 6, ciclos, de, R, -, CHOP, ., Se, le, administró, radioterapia, genital, coadyuvante, y, quimioprofilaxis, cerebral, ., A, los, 12, meses, el, paciente, se, encuentra, libre, ...]","[O, O, O, O, O, O, O, O, B-ENFERMEDAD, O, B-ENFERMEDAD, O, O, O, O, B-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...]"
2,4858,"[Para, la, realización, de, la, 1a, plasmaféresis, en, UCI, se, intenta, implantar, un, catéter, en, yugular, derecha, ,, pero, debido, a, su, dificultad, y, exceso, de, sangrado, ,, se, coloca, un, catéter, doble, luz, en, femoral, izquierda, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,2919,"[Paciente, varón, de, 69, años, ,, con, antecedente, de, gastrectomía, parcial, hace, 30, años, por, ulcus, gástrico, péptico, con, reconstrucción, de, tipo, Billroth, II, que, ingresa, por, clínica, de, ocho, días, de, evolución, consistente, en, dolor, abdominal, localizado, en, mesogastrio, junto, a, un, leve, aumento, de, las, cifras, de, amilasa, y, lipasa, séricas, (, amilasa, 550, UI, /, l, ,, lipasa, 5976, UI, /, l, ), ., La, ecografía, abdominal, mostró, la, vesícula, biliar, distendida, junto, a, una, dilatación, de, las, vías, biliares, intra, -, y, extrahepáticas, y, una, gran, formación, quística, de, paredes, finas, y, morfología, tubular, ocupando, desde, ...]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ENFERMEDAD, I-ENFERMEDAD, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, O, O, O, B-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, O, O, O, B-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, ...]"
4,3801,"[La, posición, de, las, asas, intestinales, está, alterada, por, la, presencia, de, una, anomalía, en, el, cierre, del, mesenterio, del, ciego, ., Este, presenta, un, ojal, de, unos, 3, cms, de, diámetro, localizado, inmediatamente, junto, al, ciego, ,, por, el, que, se, aprecia, el, paso, extenso, de, las, asas, del, intestino, delgado, y, de, parte, del, colon, descendente, ,, que, se, halla, dilatado, y, lateralizado, hacia, la, derecha, e, interrumpido, en, su, trayecto, por, el, paso, a, través, del, ojal, ,, prosiguiendo, hacia, el, recto, desde, dicho, ojal, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, I-ENFERMEDAD, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, 

## Preprocessing the data

In [20]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(base_model)

In [21]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

In [22]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [23]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [24]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[0, 3106, 8276, 262, 2980, 610, 299, 20165, 288, 18187, 2232, 262, 40581, 1933, 51409, 262, 1337, 19025, 797, 363, 2100, 42278, 50288, 3567, 344, 297, 883, 2839, 269, 2371, 262, 23537, 1058, 313, 2149, 3904, 2469, 50288, 6460, 453, 4258, 262, 362, 10159, 23497, 262, 7685, 8799, 262, 7998, 51945, 50288, 290, 6630, 18616, 50288, 300, 313, 4470, 416, 3055, 20307, 15549, 288, 3152, 262, 7998, 51945, 1933, 30953, 551, 2149, 16389, 30953, 47771, 453, 4258, 20307, 50288, 1793, 297, 3146, 21426, 325, 923, 262, 7998, 51945, 284, 1415, 3018, 7012, 30953, 2], [0, 551, 4902, 50288, 288, 318, 680, 1006, 50288, 10359, 4487, 262, 20165, 20307, 288, 7625, 7998, 51945, 1933, 50288, 526, 3243, 262, 574, 488, 5810, 2917, 30953, 2], [0, 452, 2385, 1006, 313, 2731, 4794, 12426, 14013, 13774, 262, 284, 9569, 4487, 30953, 18630, 27965, 17352, 23071, 224, 29, 1820, 20307, 262, 362, 50288, 617, 48678, 350, 990, 20, 2935, 290, 362, 50288, 388, 48678, 350, 990, 388, 2935, 262, 11096, 9999, 50288, 

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [25]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/5182 [00:00<?, ? examples/s]

Map:   0%|          | 0/7208 [00:00<?, ? examples/s]

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [26]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

/home/sylvia/.local/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-biomedical-clinical-es and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

In [27]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    output_dir = f"{output_path}/{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=2,
    adam_beta1=0.9,
    adam_beta2=0.999,
    num_train_epochs=20,
    weight_decay=0,
    save_strategy = "epoch",
    push_to_hub=False,
)

In [28]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [29]:
metric = load_metric("seqeval")

/tmp/ipykernel_50598/152412463.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


This metric takes list of labels for the predictions and references:

In [30]:
labels = [label_list[i] for i in datasets['train'][0][f"labels"]]
metric.compute(predictions=[labels], references=[labels])

{'ENFERMEDAD': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [31]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [32]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/home/sylvia/.local/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [33]:
trainer.evaluate()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: svassileva. Use `wandb login --relogin` to force relogin


{'eval_loss': 1.1132551431655884,
 'eval_precision': 0.005662972251435968,
 'eval_recall': 0.0489040258492708,
 'eval_f1': 0.010150536075186472,
 'eval_accuracy': 0.26073371371753296,
 'eval_runtime': 48.188,
 'eval_samples_per_second': 149.581,
 'eval_steps_per_second': 4.69}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [34]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

print(predictions)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

[[1 0 1 ... 0 0 0]
 [1 0 2 ... 0 0 0]
 [1 2 0 ... 0 0 0]
 ...
 [1 1 2 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [2 0 2 ... 0 0 0]]


{'ENFERMEDAD': {'precision': 0.005662972251435968,
  'recall': 0.0489040258492708,
  'f1': 0.010150536075186472,
  'number': 11451},
 'overall_precision': 0.005662972251435968,
 'overall_recall': 0.0489040258492708,
 'overall_f1': 0.010150536075186472,
 'overall_accuracy': 0.26073371371753296}